*Reading in Google Mobility Data: 

Note: data are read as "percent change from baseline"

This was my R code: 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import ipydatatable
import unittest

Reading in dataframe and filtering for State of Virginia

In [ ]:
#this takes a long while to run and is a huge file so I did this once and saved a smaller, csv file filtered to Virginia

#Google_Mob = pd.read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=6d352e35dcffafce")
#Google_Mob_VA = Google_Mob[(Google_Mob.country_region == "United States") & (Google_Mob.sub_region_1 == "Virginia")] #filtering for Virginia, USA
#del Google_Mob
# Google_Mob_VA.to_csv("./google_mobility_VA.csv")


In [ ]:
#sometimes you need to try to run this chunk twice
Google_Mob_VA = pd.read_csv("./google_mobility_VA.csv")


In [ ]:
Google_Mob_VA['date'] = pd.to_datetime(Google_Mob_VA['date']) #change to date format.

In [ ]:
Google_Mob_VA.columns

Renaming State, Counties

In [ ]:
Google_Mob_VA = Google_Mob_VA.rename(columns={"sub_region_1": "state", "sub_region_2": "county", "retail_and_recreation_percent_change_from_baseline": "retail", "grocery_and_pharmacy_percent_change_from_baseline":"grocery", "parks_percent_change_from_baseline": "parks","transit_stations_percent_change_from_baseline": "transit", "workplaces_percent_change_from_baseline": "workplaces", "residential_percent_change_from_baseline": "residential"}) #rename state, county columns


In [ ]:
Google_Mob_VA = Google_Mob_VA.drop(['iso_3166_2_code', 'place_id', 'country_region_code', "metro_area", "census_fips_code"], axis=1) #dropping unnecessary columns

In [ ]:
#checking
Google_Mob_VA.columns

In [ ]:
Google_Mob_VA.dtypes

In [ ]:
#using same masking technique that Wenwei employed here. 
start_date = '03-01-2020'
end_date = '09-01-2020'

mask = (Google_Mob_VA['date'] > start_date) & (Google_Mob_VA['date'] <= end_date)
#assign mask to df to return the rows with _date between our specified start/end dates

Google_Mob_VA_peakdates = Google_Mob_VA.loc[mask] #specified the date range to be within 1st 6 months of pandemic onset.
#NYTCovid_peakdates

Unit testing both the date column type and correct date range mask technique used above. 

In [ ]:
#note you have to restart the kernel every time you try this or a new unittest
class DatesTestCase(unittest.TestCase): 
    def test_1_datemask(self):
        #checking/printing date range
        print(Google_Mob_VA_peakdates.date.describe(datetime_is_numeric=True))
            # must declare global objects to use within the class
        #global Google_Mob_VA
        #global mask
        #Google_Mob_VA_peakdates = Google_Mob_VA.loc[mask]
        expected_dates_start = pd.to_datetime('02-28-2020')
        expected_dates_stop = pd.to_datetime('09-02-2020')
        #checking that date column is indeed of the correct type: date
        self.assertTrue(Google_Mob_VA_peakdates['date'].dtype == "datetime64[ns]")
        #checking that date bounds are correct: 
        print("minimum date: ",Google_Mob_VA_peakdates['date'].min())
        print("maximum date: ",Google_Mob_VA_peakdates['date'].max())
        self.assertTrue(Google_Mob_VA_peakdates['date'].min()> expected_dates_start and Google_Mob_VA_peakdates['date'].max() < expected_dates_stop)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False) 

In [ ]:
display(Google_Mob_VA_peakdates)

In [ ]:
#dropping rows with missing county data
Google_Mob_VA_peakdates = Google_Mob_VA_peakdates.dropna(subset=['county'])

In [ ]:
#Google_Mob_VA_peakdates.county.unique()
Google_Mob_VA_peakdates[Google_Mob_VA_peakdates.county == 'Albemarle County'].date

Basic plots for Sanity check

In [ ]:


plt.plot(Google_Mob_VA_peakdates[Google_Mob_VA_peakdates.county == 'Albemarle County'].date, Google_Mob_VA_peakdates[Google_Mob_VA_peakdates.county == 'Albemarle County'].retail)
plt.show()

melting by different mobility indices


state                       object
county                      object
metro_area                 float64
census_fips_code           float64
date                datetime64[ns]
retail                     float64
grocery                    float64
parks                      float64
transit                    float64
workplaces                 float64
residential                float64

Splitting mobility data into more generic groups: residential vs. non-residential

In [ ]:
Google_Mob_VA_peakdates_grouped = Google_Mob_VA_peakdates.groupby(['date','state','county'], as_index=False).mean().sort_values(["county","date"])
#mutating new column which is the average non-residential mobility. 
Google_Mob_VA_peakdates_grouped['Non_Residential'] = Google_Mob_VA_peakdates_grouped[['retail', 'grocery','parks', 'transit', 'workplaces']].mean(axis=1)
#adding a 7d moving average to non-residential smooth out the weekend effects and deal with NaN values
Google_Mob_VA_peakdates_grouped['Non_Residential_7d_Rolling'] = Google_Mob_VA_peakdates_grouped['Non_Residential'].rolling(7).mean()
#same for residential
Google_Mob_VA_peakdates_grouped['Residential_7d_Rolling'] = Google_Mob_VA_peakdates_grouped['residential'].rolling(7).mean()
Google_Mob_VA_peakdates_grouped

In [ ]:
#.reset_index() is equivalent to ungroup()
Google_Mob_VA_peakdates_clean = Google_Mob_VA_peakdates_grouped.reset_index()
Google_Mob_VA_peakdates_clean['date'] = pd.to_datetime(Google_Mob_VA_peakdates_clean['date']) #change to date format.
Google_Mob_VA_peakdates_clean = Google_Mob_VA_peakdates_clean.sort_values(["county","date"])
#Google_Mob_VA_peakdates_clean['Non_Residential'] = Google_Mob_VA_peakdates_grouped[['retail', 'grocery','parks', 'transit', 'workplaces']].mean(axis=1)

In [ ]:

#Google_Mob_VA_peakdates_clean
#ipydatatable.InteractiveTable(table =Google_Mob_VA_peakdates_clean)
Google_Mob_VA_peakdates_clean

Measuring minimum non-residential mobility over time for each county. 

In [ ]:

county_minimums = Google_Mob_VA_peakdates_clean.groupby('county', as_index=False)['Non_Residential_7d_Rolling'].min()
county_minimums = county_minimums.reindex(columns=county_minimums.columns)
county_minimums.columns = ['county', 'Minimum_Non_Residential_7d_Rolling']
county_minimums
Google_Mob_VA_peakdates_clean = Google_Mob_VA_peakdates_clean.merge(county_minimums, on='county', how='left')
Google_Mob_VA_peakdates_clean['Low_Point_Mobility'] = 0
Google_Mob_VA_peakdates_clean.Low_Point_Mobility[Google_Mob_VA_peakdates_clean.Minimum_Non_Residential_7d_Rolling == Google_Mob_VA_peakdates_clean.Non_Residential_7d_Rolling] =1

Google_Mob_VA_peakdates_clean

In [ ]:
Google_Mob_VA_peakdates_clean[Google_Mob_VA_peakdates_clean.Low_Point_Mobility==1].sort_values('county').head(130)

In [ ]:

#writing cleaned dataset to csv
Google_Mob_VA_peakdates_clean.to_csv("./Google_Mob_VA_peakdates_clean.csv")

#writing county minimums dataset to csv
county_minimums.to_csv("./county_minimums.csv")


In [ ]:
Google_Mob_VA_peakdates_clean_melted_all = pd.melt(Google_Mob_VA_peakdates_clean, id_vars = ['state','county', 'date'], value_vars=['retail', 'grocery','parks', 'transit', 'workplaces','residential','Non_Residential'])
Google_Mob_VA_peakdates_clean_melted_all
Google_Mob_VA_peakdates_clean_melted_residential = pd.melt(Google_Mob_VA_peakdates_clean, id_vars = ['state','county', 'date'], value_vars=['residential','Non_Residential'])
Google_Mob_VA_peakdates_clean_melted_residential
Google_Mob_VA_peakdates_clean_melted_residential_rollmean = pd.melt(Google_Mob_VA_peakdates_clean, id_vars = ['state','county', 'date'], value_vars=['Non_Residential', 'Non_Residential_7d_Rolling'])
Google_Mob_VA_peakdates_clean_melted_residential_rollmean


In [ ]:
g = Google_Mob_VA_peakdates_clean_melted_all[Google_Mob_VA_peakdates_clean_melted_all.county == 'Albemarle County'].pivot(index="date", columns=["variable"], values="value").plot()
g= g.legend(loc='best',bbox_to_anchor=(0.8, 0., 0.6, 0.8))
plt.ylabel("Percent Change from Baseline")
plt.xlabel("")
plt.show()


Showing average Residential vs. Non-Residential Mobility for single county (Albemarle) over time. Note the significant effect of weekends leading to spikes in mobility. Note there are a good amount of missing data for residential mobility. Probably related to privacy scrubbing. 

In [ ]:
g = Google_Mob_VA_peakdates_clean_melted_residential[Google_Mob_VA_peakdates_clean_melted_residential.county == 'Albemarle County'].pivot(index="date", columns=["variable"], values="value").plot()
g= g.legend(loc='best',bbox_to_anchor=(0.8, 0., 0.6, 0.8))
plt.ylabel("Percent Change from Baseline")
plt.xlabel("")
plt.show()


Now showing raw Non-Residential Data as a 7-day rolling mean. I think this looks a lot better. 

In [ ]:
g = Google_Mob_VA_peakdates_clean_melted_residential_rollmean[Google_Mob_VA_peakdates_clean_melted_residential_rollmean.county == 'Albemarle County'].pivot(index="date", columns=["variable"], values="value").plot()
g= g.legend(loc='best',bbox_to_anchor=(0.8, 0., 0.6, 0.8))
plt.ylabel("Percent Change from Baseline")
plt.xlabel("")
plt.show()






In [ ]:
# Create a grid : initialize it

sns.set_theme(style="ticks")

dots = sns.load_dataset("dots")

# Plot the lines on two facets
g=sns.relplot(
    data=Google_Mob_VA_peakdates_clean_melted_residential_rollmean,
    x="date", y="value",
    hue="variable",  col="county",
    kind="line",  col_wrap =7,
    height=5, aspect=.75, facet_kws=dict(sharex=True),
)
g = g.set_titles(col_template = '{col_name}')
g.title("Google Mobility Changes by County")
#g.ylabel()
#g.xlabel("")
g.set_axis_labels(x_var="Date", y_var="Percent Change from Baseline")
g.show()
g.savefig("mobility_plot.png")
#g.show()
#sns.*plot*?



writing melted dataset to csv as well

In [ ]:
Google_Mob_VA_peakdates_clean.to_csv("./Google_Mob_VA_peakdates_clean.csv")
Google_Mob_VA_peakdates_clean

Now creatining state-wide dataset for mobility.

In [ ]:
ipydatatable.InteractiveTable(table =Google_Mob_VA_peakdates_clean)

To dos: 
-create dataset and plot for entire state of virginia
-calculate largest reduction in mobility by county

In [ ]:
#grouping by just state this time. 
Google_Mob_VA_peakdates_grouped_all_state = Google_Mob_VA_peakdates.groupby(['state',"date"], as_index=False).mean().sort_values(["date"])
#mutating the new average non-residential mobility. 
Google_Mob_VA_peakdates_grouped_all_state['Non_Residential'] = Google_Mob_VA_peakdates_grouped_all_state[['retail', 'grocery','parks', 'transit', 'workplaces']].mean(axis=1)
#adding a 7d moving average to non-residential smooth out the weekend effects and deal with NaN values
Google_Mob_VA_peakdates_grouped_all_state['Non_Residential_7d_Rolling'] = Google_Mob_VA_peakdates_grouped_all_state['Non_Residential'].rolling(7).mean()
#same for residential
Google_Mob_VA_peakdates_grouped_all_state['Residential_7d_Rolling'] = Google_Mob_VA_peakdates_grouped_all_state['residential'].rolling(7).mean()
Google_Mob_VA_peakdates_grouped_all_state = Google_Mob_VA_peakdates_grouped_all_state.reset_index()

QUESTION 1: In Virginia, when is the largest trough in mobility? 

Plotting whole state rollmean

In [ ]:
Google_Mob_VA_peakdates_grouped_all_state

In [ ]:
plt.plot(Google_Mob_VA_peakdates_grouped_all_state.date, Google_Mob_VA_peakdates_grouped_all_state.Non_Residential_7d_Rolling)
#show vertical line at 2020-04-15 00:00:00 and horizontal line at -26.047695
plt.axhline(y=-26.047695, linewidth=1, color = 'r')
plt.axvline(x= pd.to_datetime("2020-04-15", infer_datetime_format=True), linewidth=1, color = 'r')
plt.annotate('April 15, 2020 Maximum \n26% Reduction: Non-Residential Mobility',
            xy=(0.235, 0.04),
            xycoords='axes fraction',
            xytext=(0.33, 0.25),
            arrowprops=
                dict(facecolor='black', shrink=0.05),
                horizontalalignment='left',
                verticalalignment='top')
plt.title("Early Pandemic Mobility Changes in Virginia")
plt.ylabel("%Change Mobility vs. Pre-Pandemic, 7d Rolling Mean)")
plt.savefig("minimum_statewide_mobility.png")
plt.show()

#Google_Mob_VA_peakdates_grouped_all_state.columns

QUESTION 1 Cont.: Calculating the actual date of lowest 7d average non-residentialmobility

In [ ]:
Google_Mob_VA_peakdates_grouped_all_state.loc[Google_Mob_VA_peakdates_grouped_all_state['Non_Residential_7d_Rolling'].idxmin()]
#df.loc[df['Score'].idxmax()]
